### Email Agent

- Authenticates user

Only then are they allowed into the "inbox"
Dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded

- Checks "inbox"

Email in tool

- Sends emails

Human in the loop

In [8]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [9]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [10]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool 
def authenticate( email: str, password: str, runtime: ToolRuntime ) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command( update= {
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [11]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [12]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [13]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_ollama import ChatOllama
# Load the model
model = ChatOllama(model="qwen3:1.7b", 
                   temperature=0,
                   validate_model_on_init=True,
                   seed=42)

agent = create_agent(
    model,
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )

In [14]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="draft 1")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

The provided tool requires an email and password to authenticate the user. Please provide those details so I can assist further.


In [15]:
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='draft 1', additional_kwargs={}, response_metadata={}, id='56ffdfde-79ef-42d2-b934-14d48d32e0d2'),
              AIMessage(content='The provided tool requires an email and password to authenticate the user. Please provide those details so I can assist further.', additional_kwargs={}, response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-12-30T12:41:02.0483255Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6714669400, 'load_duration': 4567210600, 'prompt_eval_count': 159, 'prompt_eval_duration': 491012700, 'eval_count': 103, 'eval_duration': 1425458300, 'logprobs': None, 'model_name': 'qwen3:1.7b', 'model_provider': 'ollama'}, id='lc_run--019b6f46-9524-7520-89c5-a3dedc7949aa-0', usage_metadata={'input_tokens': 159, 'output_tokens': 103, 'total_tokens': 262})]}


In [18]:
# print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

```json
{
  "name": "authenticate",
  "parameters": {
    "email": "user@example.com",
    "password": "password123"
  }
}
```
